# LOGISTIC REGRESSION

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from nltk.corpus import stopwords
import joblib

In [10]:
from sklearn.datasets import load_iris
data=pd.read_csv("C:\\Users\\ajlad\Downloads\\bbc_data.csv")

In [11]:
data

,data,labels
0,Musicians to tackle US red tape Musicians gro...,entertainment
1,"U2s desire to be number one U2, who have won ...",entertainment
2,Rocker Doherty in on-stage fight Rock singer ...,entertainment
3,Snicket tops US box office chart The film ada...,entertainment
4,"Oceans Twelve raids box office Oceans Twelve,...",entertainment
...,...,...
2220,Warning over Windows Word files Writing a Mic...,tech
2221,Fast lifts rise into record books Two high-sp...,tech
2222,Nintendo adds media playing to DS Nintendo is...,tech
2223,Fast moving phone viruses appear Security fir...,tech


In [13]:
w2v=joblib.load("w2v.pkl")


In [14]:

# Define the preprocessing function
def preprocess_text(text):
    # Ensure text is a string
    if not isinstance(text, str):
        return []
    
    # Lowercase the text
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize and remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in text.split() if word not in stop_words]
    
    return tokens
data['data'] = data['data'].apply(preprocess_text)

In [45]:
def word_vectorizer(doc, model, num_features):
    feature_vector = np.zeros((num_features,), dtype="float32")
    num_words = 0
    for word in doc:
        if word in model.wv:
            num_words += 1
            feature_vector = np.add(feature_vector, model.wv[word])
    if num_words > 0:
        feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

# Apply word vectorization to each document
data['vectors'] = data['data'].apply(lambda x: word_vectorizer(x, w2v, 100))

# Example to check the vector for the first document
print(data['vectors'].head(1))

0    [0.12256169, 0.24330522, 0.03752677, 0.2188870...
Name: vectors, dtype: object


In [65]:
data.head()

,data,labels,vectors
0,"[musicians, tackle, us, red, tape, musicians, ...",entertainment,"[0.12256169, 0.24330522, 0.03752677, 0.2188870..."
1,"[u2s, desire, number, one, u2, three, prestigi...",entertainment,"[0.15952033, 0.2144658, 0.08485847, 0.13640614..."
2,"[rocker, doherty, onstage, fight, rock, singer...",entertainment,"[0.18606852, 0.20529948, 0.10971801, 0.0996567..."
3,"[snicket, tops, us, box, office, chart, film, ...",entertainment,"[0.21293125, 0.17197491, 0.15845276, 0.0451965..."
4,"[oceans, twelve, raids, box, office, oceans, t...",entertainment,"[0.19102304, 0.17561013, 0.13803297, 0.0702821..."


In [53]:
from sklearn.linear_model import LogisticRegression
logRe=LogisticRegression()

In [54]:
from sklearn.model_selection import train_test_split

In [55]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)

In [56]:
x = np.array(data['vectors'].tolist())  # Ensure this is a 2D array
y = np.array(data['labels'])

In [57]:
logRe.fit(x_train,y_train)

d:\codes\env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [58]:
logRe.predict(x_test)

array(['business', 'entertainment', 'entertainment', 'business',
       'politics', 'business', 'politics', 'sport', 'sport',
       'entertainment', 'entertainment', 'tech', 'tech', 'politics',
       'business', 'entertainment', 'politics', 'sport', 'business',
       'politics', 'sport', 'tech', 'tech', 'tech', 'business',
       'politics', 'politics', 'entertainment', 'politics',
       'entertainment', 'sport', 'politics', 'politics', 'business',
       'tech', 'sport', 'business', 'business', 'business', 'sport',
       'sport', 'sport', 'politics', 'politics', 'tech', 'sport',
       'politics', 'sport', 'sport', 'politics', 'business', 'tech',
       'politics', 'tech', 'sport', 'politics', 'business', 'tech',
       'tech', 'tech', 'business', 'entertainment', 'business', 'tech',
       'business', 'entertainment', 'tech', 'business', 'politics',
       'business', 'tech', 'entertainment', 'tech', 'entertainment',
       'tech', 'entertainment', 'business', 'tech', 'entertain

In [59]:
logRe.score(x_train,y_train)

0.7117977528089887

In [60]:
logRe.intercept_

array([ 2.03001142,  6.16599866, -4.93139788, -0.35978602, -2.90482618])

In [64]:
test=" high-octane superhero film that has already broken several box office records. The movie, directed by a renowned filmmaker known for his previous hits, features an all-star cast that includes some of the biggest names in the industry. Critics are praising the film for its groundbreaking special effects and engaging storyline. Fans are flocking to theaters in droves, eager to experience the action-packed spectacle on the big screen. Additionally, the film's soundtrack, featuring tracks from popular artists, has also topped the music charts, further solidifying its place in popular culture."
t_v=preprocess_text(test)
v=word_vectorizer(t_v, w2v, 100).reshape(1,-1)
y = logRe.predict(v)[0]
y

'entertainment'